# T1552 - Unsecured Credentials
Adversaries may search compromised systems to find and obtain insecurely stored credentials. These credentials can be stored and/or misplaced in many locations on a system, including plaintext files (e.g. [Bash History](https://attack.mitre.org/techniques/T1552/003)), operating system or application-specific repositories (e.g. [Credentials in Registry](https://attack.mitre.org/techniques/T1552/002)), or other specialized files/artifacts (e.g. [Private Keys](https://attack.mitre.org/techniques/T1552/004)).

## Atomic Tests

### Atomic Test #1 - AWS - Retrieve EC2 Password Data using stratus
This atomic runs an API call GetPasswordData from a role that does not have permission to do so. This simulates an attacker attempting to retrieve RDP passwords on a high number of Windows EC2 instances. This atomic test leverages a tool called stratus-red-team built by DataDog (https://github.com/DataDog/stratus-red-team). Stratus Red Team is a self-contained binary. You can use it to easily detonate offensive attack techniques against a live cloud environment. Ref: https://stratus-red-team.cloud/attack-techniques/AWS/aws.credential-access.ec2-get-password-data/

**Supported Platforms:** linux, macos
#### Dependencies:  Run with `sh`!
##### Description: Stratus binary must be present at the (#{stratus_path}/stratus)

##### Check Prereq Commands:
```sh
if [ -f $PathToAtomicsFolder/T1552/src/stratus ]; then exit 0; else exit 1; fi;

```
##### Get Prereq Commands:
```sh
if [ "$(uname)" == "Darwin" ]
then DOWNLOAD_URL=$(curl -s https://api.github.com/repos/DataDog/stratus-red-team/releases/latest | grep browser_download_url | grep Darwin_x86_64 | cut -d '"' -f 4); wget -q -O $PathToAtomicsFolder/T1552/src/stratus-red-team-latest.tar.gz $DOWNLOAD_URL
  tar -xzvf $PathToAtomicsFolder/T1552/src/stratus-red-team-latest.tar.gz --directory $PathToAtomicsFolder/T1552/src/
elif [ "$(expr substr $(uname) 1 5)" == "Linux" ]
then DOWNLOAD_URL=$(curl -s https://api.github.com/repos/DataDog/stratus-red-team/releases/latest | grep browser_download_url | grep linux_x86_64 | cut -d '"' -f 4) 
  wget -q -O $PathToAtomicsFolder/T1552/src/stratus-red-team-latest.tar.gz $DOWNLOAD_URL
  tar -xzvf $PathToAtomicsFolder/T1552/src/stratus-red-team-latest.tar.gz --directory $PathToAtomicsFolder/T1552/src/
fi

```
##### Description: Check if ~/.aws/credentials file has a default stanza is configured

##### Check Prereq Commands:
```sh
cat ~/.aws/credentials | grep "default"

```
##### Get Prereq Commands:
```sh
echo Please install the aws-cli and configure your AWS defult profile using: aws configure

```

In [ ]:
Invoke-AtomicTest T1552 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
export AWS_REGION=us-west-2 
cd $PathToAtomicsFolder/T1552/src
echo "starting warmup"
./stratus warmup aws.credential-access.ec2-get-password-data
echo "starting detonate"
./stratus detonate aws.credential-access.ec2-get-password-data --force
```

In [ ]:
Invoke-AtomicTest T1552 -TestNumbers 1

## Detection
While detecting adversaries accessing credentials may be difficult without knowing they exist in the environment, it may be possible to detect adversary use of credentials they have obtained. Monitor the command-line arguments of executing processes for suspicious words or regular expressions that may indicate searching for a password (for example: password, pwd, login, secure, or credentials). See [Valid Accounts](https://attack.mitre.org/techniques/T1078) for more information.

Monitor for suspicious file access activity, specifically indications that a process is reading multiple files in a short amount of time and/or using command-line arguments  indicative of searching for credential material (ex: regex patterns). These may be indicators of automated/scripted credential access behavior.

Monitoring when the user's <code>.bash_history</code> is read can help alert to suspicious activity. While users do typically rely on their history of commands, they often access this history through other utilities like "history" instead of commands like <code>cat ~/.bash_history</code>.

Additionally, monitor processes for applications that can be used to query the Registry, such as [Reg](https://attack.mitre.org/software/S0075), and collect command parameters that may indicate credentials are being searched. Correlate activity with related suspicious behavior that may indicate an active intrusion to reduce false positives.

## Shield Active Defense
### Decoy Credentials 
 Create user credentials that are used for active defense purposes. 

 Seed a target system with credentials (such as username/password, browser tokens, and other forms of authentication data) for the purpose of engagement. Decoy credentials can be planted in many locations and leveraged in a variety of ways.
#### Opportunity
In order to prolong an adversary engagement operation or enable detections, there is an opportunity to introduce credentials to an adversary that you want them to collect and use.
#### Use Case
A defender can plant decoy credentials across an array of locations to increase the chances of an adversary finding and using them.
#### Procedures
Create user credentials for a decoy account, such as 'User ABC'.  Store those credentials in the browser and other places on the system to see if an adversary attempts to harvest them.